In [11]:
import hopsworks
from hsfs.client.exceptions import RestAPIError

# Establish connection to Hopsworks
project = hopsworks.login(
    api_key_value="dYTVrTVvbj6Qw82i.YGKHdS9snQYFgOADJIvLdvZ2n2S5BxIAOtvPUEmAyd56bvaG6xhhGyNM3nYbexaP"
)

# Access the Feature Store
fs = project.get_feature_store()

# List of feature groups and their versions
feature_groups = [
    {"name": "city_weather", "version": 1},
    {"name": "drivers", "version": 1},
    {"name": "routes", "version": 1},
    {"name": "routes_weather", "version": 1},
    {"name": "traffic", "version": 1},  # Corrected typo
    {"name": "truck_schedule", "version": 1},
    {"name": "trucks", "version": 1},
]
# Initialize an empty dictionary to hold DataFrames for each feature group
feature_group_data = {}

# Loop over the list of feature groups and download them
for fg in feature_groups:
    try:
        # Retrieve the feature group by its name and version
        feature_group = fs.get_feature_group(fg['name'], version=fg['version'])
        df = feature_group.read()  # Read the feature group as a DataFrame
        
        # Save the DataFrame in the dictionary with the feature group's name as the key
        feature_group_data[fg['name']] = df

        print(f"Downloaded feature group: {fg['name']} (version {fg['version']})")
        print(df.head())  # Optionally display the first few rows of the DataFrame

    except RestAPIError as e:
        print(f"Error downloading feature group: {fg['name']} (version {fg['version']})")
        print(e)

# After the loop, you will have each feature group stored in the `feature_group_data` dictionary
# Access each DataFrame like this: feature_group_data['city_weather'], etc.



Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1044630
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.51s) 
Downloaded feature group: city_weather (version 1)
   index_column     city_id  temp  wind_speed    description  precip  \
0         26374  C-19236709    55          10          Sunny     0.0   
1         37883  C-e6dcda92    19          17          Sunny     0.0   
2         21614  C-419cd14c    57           9          Sunny     0.0   
3          9578  C-ef47bdcd    36           7  Partly cloudy     0.0   
4         29488  C-34ae77b0    79           6          Clear     0.0   

   humidity  visibility  pressure  chanceofrain  chanceoffog  chanceofsnow  \
0        62           6      1001             0            0             0   
1        74           6      1027             0  

In [12]:
print(feature_group_data.keys())


dict_keys(['city_weather', 'drivers', 'routes', 'routes_weather', 'traffic', 'truck_schedule', 'trucks'])


In [13]:
city_weather = feature_group_data['city_weather']
drivers = feature_group_data['drivers']
routes = feature_group_data['routes']
routes_weather = feature_group_data['routes_weather']
traffic = feature_group_data['traffic'] 
truck_schedule = feature_group_data['truck_schedule']
trucks = feature_group_data['trucks']


In [16]:
def assign_feature_group_data(feature_group_data, table_names):
    # Dynamically assign the DataFrame from feature_group_data to the variable names provided in table_names
    for table_name in table_names:
        globals()[table_name] = feature_group_data[table_name]
        print(f"Assigned {table_name} to variable.")
        
# List of table names to be assigned
table_names = ['city_weather', 'routes_weather', 'traffic', 'truck_schedule', 'trucks']

# Call the function to assign feature group data to variables
assign_feature_group_data(feature_group_data, table_names)

# Now you can access the variables dynamically assigned
print(truck_schedule.head())        

# 'drivers_table', 'routes_table'


Assigned city_weather to variable.
Assigned routes_weather to variable.
Assigned traffic to variable.
Assigned truck_schedule to variable.
Assigned trucks to variable.
   index_column  truck_id    route_id       departure_date  \
0          5832  46155960  R-c3d9f83b  2019-02-03 07:00:00   
1          1825  23024395  R-f782724c  2019-01-10 07:00:00   
2          9186  23608280  R-4d09a561  2019-01-17 07:00:00   
3          3063  28641704  R-a55547f0  2019-01-16 07:00:00   
4          9578  13426754  R-006548e0  2019-02-10 07:00:00   

               estimated_arrival  delay                event_time  
0  2019-02-03 08:31:48.000000000      1 2024-09-17 00:00:00+00:00  
1  2019-01-10 11:55:48.000000000      1 2024-09-17 00:00:00+00:00  
2  2019-01-19 14:35:24.000000000      0 2024-09-17 00:00:00+00:00  
3  2019-01-16 19:36:00.000000000      1 2024-09-17 00:00:00+00:00  
4  2019-02-11 23:01:12.000000000      1 2024-09-17 00:00:00+00:00  


In [25]:

dfs = {
    'city_weather': city_weather,
    'drivers': drivers,
    'routes': routes,
    'routes_weather': routes_weather,
    'traffic': traffic,
    'truck_schedule': truck_schedule,
    'trucks': trucks,
}


def drop_columns_from_tables(dfs, columns_to_drop):
    """
    Drops specified columns from all DataFrames in the provided dictionary.
    
    Parameters:
    - dfs (dict): A dictionary where the keys are DataFrame names and the values are pandas DataFrames.
    - columns_to_drop (list): A list of column names to be dropped from each DataFrame.
    
    Returns:
    - dict: A dictionary of DataFrames with the specified columns removed.
    """
    
    # Loop through each DataFrame in the dictionary
    for table_name, df in dfs.items():
        # Check if the columns exist in the DataFrame before dropping
        cols_to_drop = [col for col in columns_to_drop if col in df.columns]
        
        # If there are columns to drop, drop them
        if cols_to_drop:
            dfs[table_name] = df.drop(columns=cols_to_drop, inplace=True)
            print(f"Dropped columns {cols_to_drop} from {table_name}.")
        else:
            print(f"No matching columns to drop in {table_name}.")
    
    return dfs

# Example usage:
columns_to_drop = ['index_column', 'event_time']  # Example columns that might be present in multiple tables
feature_group_data_cleaned = drop_columns_from_tables(feature_group_data, columns_to_drop)

# Access the cleaned DataFrames as needed
feature_group_data_cleaned['city_weather'].head()


No matching columns to drop in city_weather.
No matching columns to drop in drivers.
No matching columns to drop in routes.
No matching columns to drop in routes_weather.
No matching columns to drop in traffic.
No matching columns to drop in truck_schedule.
No matching columns to drop in trucks.


,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder,datetime
0,C-19236709,55,10,Sunny,0.0,62,6,1001,0,0,0,0,2019-02-14 13:00:00+00:00
1,C-e6dcda92,19,17,Sunny,0.0,74,6,1027,0,0,0,0,2019-01-24 11:00:00+00:00
2,C-419cd14c,57,9,Sunny,0.0,70,6,1029,0,0,0,0,2019-01-09 06:00:00+00:00
3,C-ef47bdcd,36,7,Partly cloudy,0.0,53,6,1005,0,0,0,0,2019-01-06 07:00:00+00:00
4,C-34ae77b0,79,6,Clear,0.0,45,6,1016,0,0,0,0,2019-01-12 20:00:00+00:00


In [21]:
def drop_duplicates_from_dfs(dfs, columns_to_check):
    for df_name, df in dfs.items():
        if df_name in columns_to_check:
            subset_columns = columns_to_check[df_name]
            
            # Check if all columns in subset_columns exist in the DataFrame
            missing_columns = [col for col in subset_columns if col not in df.columns]
            if missing_columns:
                print(f"Warning: DataFrame {df_name} does not contain the columns: {missing_columns}")
                continue  # Skip this DataFrame and move to the next one
            
            before_dropping = df.shape[0]  # Count the number of rows before dropping duplicates
            df.drop_duplicates(subset=subset_columns, inplace=True)
            after_dropping = df.shape[0]  # Count the number of rows after dropping duplicates
            print(f"Dropped {before_dropping - after_dropping} duplicate rows from {df_name} based on columns: {subset_columns}")
        else:
            print(f"No duplicate check columns provided for {df_name}")

# Dictionary where the key is the DataFrame name, and the value is the subset of columns to check for duplicates
columns_to_check = {
    'city_weather': ['city_id', 'datetime'],
    'routes_weather': ['route_id', 'date'],
    'trucks': ['truck_id'],
    'drivers': ['driver_id'],
    'routes': ['route_id', 'destination_id', 'origin_id'],
    'truck_schedule': ['truck_id', 'route_id', 'departure_date']
}

# Call the function to drop duplicates from the DataFrames
drop_duplicates_from_dfs(dfs, columns_to_check)

# Check the result by printing the modified DataFrames
for df_name, df in dfs.items():
    print(f"\n{df_name} DataFrame after dropping duplicates:")
    print(df)


Dropped 707 duplicate rows from city_weather based on columns: ['city_id', 'datetime']
Dropped 0 duplicate rows from drivers based on columns: ['driver_id']
Dropped 0 duplicate rows from routes based on columns: ['route_id', 'destination_id', 'origin_id']
Dropped 0 duplicate rows from routes_weather based on columns: ['route_id', 'date']
No duplicate check columns provided for traffic
Dropped 0 duplicate rows from truck_schedule based on columns: ['truck_id', 'route_id', 'departure_date']
Dropped 0 duplicate rows from trucks based on columns: ['truck_id']

city_weather DataFrame after dropping duplicates:
       index_column     city_id  temp  wind_speed        description  precip  \
0             26374  C-19236709    55          10              Sunny     0.0   
1             37883  C-e6dcda92    19          17              Sunny     0.0   
2             21614  C-419cd14c    57           9              Sunny     0.0   
3              9578  C-ef47bdcd    36           7      Partly cloud

In [24]:
city_weather.head(5)

,index_column,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder,datetime,event_time
0,26374,C-19236709,55,10,Sunny,0.0,62,6,1001,0,0,0,0,2019-02-14 13:00:00+00:00,2024-09-17 00:00:00+00:00
1,37883,C-e6dcda92,19,17,Sunny,0.0,74,6,1027,0,0,0,0,2019-01-24 11:00:00+00:00,2024-09-17 00:00:00+00:00
2,21614,C-419cd14c,57,9,Sunny,0.0,70,6,1029,0,0,0,0,2019-01-09 06:00:00+00:00,2024-09-17 00:00:00+00:00
3,9578,C-ef47bdcd,36,7,Partly cloudy,0.0,53,6,1005,0,0,0,0,2019-01-06 07:00:00+00:00,2024-09-17 00:00:00+00:00
4,29488,C-34ae77b0,79,6,Clear,0.0,45,6,1016,0,0,0,0,2019-01-12 20:00:00+00:00,2024-09-17 00:00:00+00:00


In [26]:
feature_group_data_cleaned['city_weather']

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder,datetime
0,C-19236709,55,10,Sunny,0.0,62,6,1001,0,0,0,0,2019-02-14 13:00:00+00:00
1,C-e6dcda92,19,17,Sunny,0.0,74,6,1027,0,0,0,0,2019-01-24 11:00:00+00:00
2,C-419cd14c,57,9,Sunny,0.0,70,6,1029,0,0,0,0,2019-01-09 06:00:00+00:00
3,C-ef47bdcd,36,7,Partly cloudy,0.0,53,6,1005,0,0,0,0,2019-01-06 07:00:00+00:00
4,C-34ae77b0,79,6,Clear,0.0,45,6,1016,0,0,0,0,2019-01-12 20:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38835,C-2aaf0e1a,23,3,Patchy light snow,0.0,61,6,1027,0,0,0,0,2019-01-13 23:00:00+00:00
38836,C-b6e04c88,30,6,Patchy light snow,0.0,87,6,1019,0,0,0,0,2019-01-26 22:00:00+00:00
38837,C-c7cacd1d,52,11,Sunny,0.0,79,6,1011,0,0,0,0,2019-02-04 16:00:00+00:00
38838,C-2c9e75ef,57,17,Partly cloudy,0.0,88,6,1009,0,0,0,0,2019-01-06 22:00:00+00:00
